In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.DRMM.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5375.07it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4976.14it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3267290.60it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10697.51it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5008.32it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 372135.72it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 4685.90it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f1325d96898>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f1325d22668>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
histgram_callback = mz.dataloader.callbacks.Histogram(
    embedding_matrix, bin_size=30, hist_mode='LCH'
)

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    callbacks=[histgram_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[histgram_callback]
)

In [8]:
padding_callback = mz.models.DRMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.DRMM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix
model.params['hist_bin_size'] = 30
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMM(
  (embedding): Embedding(30058, 300)
  (attention): Attention(
    (linear): Linear(in_features=300, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=30, out_features=10, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9018023


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-255 Loss-2.328]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5309 - normalized_discounted_cumulative_gain@5(0.0): 0.5872 - mean_average_precision(0.0): 0.5562



[Iter-510 Loss-2.050]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5704 - normalized_discounted_cumulative_gain@5(0.0): 0.6305 - mean_average_precision(0.0): 0.592



[Iter-765 Loss-1.845]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5851 - normalized_discounted_cumulative_gain@5(0.0): 0.6378 - mean_average_precision(0.0): 0.6018



[Iter-1020 Loss-1.637]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5826 - normalized_discounted_cumulative_gain@5(0.0): 0.6313 - mean_average_precision(0.0): 0.595



[Iter-1275 Loss-1.457]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.606 - normalized_discounted_cumulative_gain@5(0.0): 0.6541 - mean_average_precision(0.0): 0.6186



[Iter-1530 Loss-1.339]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5953 - normalized_discounted_cumulative_gain@5(0.0): 0.645 - mean_average_precision(0.0): 0.6136



[Iter-1785 Loss-1.261]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5966 - normalized_discounted_cumulative_gain@5(0.0): 0.6469 - mean_average_precision(0.0): 0.6069



[Iter-2040 Loss-1.223]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.596 - normalized_discounted_cumulative_gain@5(0.0): 0.6429 - mean_average_precision(0.0): 0.6075



[Iter-2295 Loss-1.186]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6026 - normalized_discounted_cumulative_gain@5(0.0): 0.6511 - mean_average_precision(0.0): 0.6111



[Iter-2550 Loss-1.169]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6056 - normalized_discounted_cumulative_gain@5(0.0): 0.6475 - mean_average_precision(0.0): 0.6094

Cost time: 10303.24234700203s
